In [16]:
import pandas as pd

import boto3
import pandas as pd
from sagemaker import get_execution_role

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import MultiLabelBinarizer
import category_encoders as ce

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

In [6]:
# Dataset

role = get_execution_role()
bucket='sagemaker-tagger'
data_key1 = 'Master_emails.csv'
data_location1 = 's3://{}/{}'.format(bucket, data_key1)

df = pd.read_csv(data_location1)

In [59]:
# def clean_categories(text):
#     text = text.replace("Promotions", '')
#     text = text.replace("'", '')
#     text = text.strip("[").strip("]")
#     text = text.strip("'").strip(",").strip(" ")
#     text = text.split(",")
#     return text

# emails['tags'] = emails['tags'].apply(clean_categories)
# emails.head()

,keys,from/to,subject,body,tags
0,809,Google <no-reply@accounts.google.com>,App password created,--00000000000002379205972971fd Content-Type: t...,[Security]
1,808,Google <no-reply@accounts.google.com>,2-Step Verification turned on,--000000000000d50ed3059729649c Content-Type: t...,[Security]
2,807,"""Medium Daily Digest"" <noreply@medium.com>",How To Wake Up at 5 A.M. Every Day | Bryan Ye ...,--337b1df667177cb513899ad44e31fb8d4f8637575996...,[Productivity]
3,806,"""no-reply-aws@amazon.com"" <no-reply-aws@amazon...",RE:[CASE 6570793521] Limit Increase: SageMaker,------=_Part_26869745_214056034.1573352866590 ...,[Productivity]
4,805,"""Medium Daily Digest"" <noreply@medium.com>",Multiclass Text Classification From Start To F...,--c4169d46c40775d3344614c9be2f1235565dfa74c04d...,[Productivity]


In [86]:
# Functions for checking NLP measures of sentiment

# def polarity_txt(text):
#     return TextBlob(text).sentiment[0]

# def subj_txt(text):
#     return  TextBlob(text).sentiment[1]

# def len_txt(text):
#     if len(text.split())>0:
#          return len(set(clean_txt(text).split()))/ len(text.split())
#     else:
#          return 0
        
# Implement the sentiment features

# emails['body'] = emails['body'].apply(str)

# emails['polarity'] = emails['body'].apply(polarity_txt)

# emails['subjectivity'] = emails['body'].apply(subj_txt)

# emails['len'] = emails['body'].apply(len_txt)

# emails.head()

In [8]:
mlb = MultiLabelBinarizer()
mlb_result = mlb.fit_transform([str(df.loc[i,'Tags']).split(', ') for i in range(len(df))])
mlb_df = pd.DataFrame(mlb_result,columns=list(mlb.classes_))

In [9]:
mlb_df.columns

Index(['Entertainment', 'Events', 'Finance', 'Other', 'Personal',
       'Productivity', 'Security', 'Shopping', 'Social', 'Travel', 'nan'],
      dtype='object')

In [11]:
df['Productivity'] = mlb_df['Productivity']
df['Entertainment'] = mlb_df['Entertainment']
df['Finance'] = mlb_df['Finance']
df['Other'] = mlb_df['Other']
df['Events'] = mlb_df['Events']
df['Travel'] = mlb_df['Travel']
df['Shopping'] = mlb_df['Shopping']
df['Social'] = mlb_df['Social']
df['Personal'] = mlb_df['Personal']

In [13]:
print(df.shape)
df.dropna(inplace=True)
print(df.shape)

(11550, 16)
(11446, 16)


In [26]:
df['Text'] = df['From'] + ' ' + df['Subject'] + ' ' + df['Message']
df['First_Tag'] = [x[0:3] for x in df['Tags']]

In [27]:
X = df['Text']
y = df['First_Tag']

0        Fin
1        Fin
2        Pro
3        Pro
4        Pro
5        Fin
6        Fin
7        Pro
8        Pro
9        Fin
10       Pro
11       Pro
12       Pro
13       Pro
14       Fin
15       Ent
16       Ent
17       Fin
18       Fin
19       Pro
20       Fin
21       Fin
22       Fin
23       Pro
24       Fin
25       Pro
26       Fin
27       Pro
28       Fin
29       Fin
        ... 
11520    Pro
11521    Ent
11522    Per
11523    Pro
11524    Pro
11525    Per
11526    Soc
11527    Per
11528    Pro
11529    Pro
11530    Pro
11531    Pro
11532    Pro
11533    Pro
11534    Pro
11535    Ent
11536    Pro
11537    Eve
11538    Ent
11539    Per
11540    Pro
11541    Pro
11542    Pro
11543    Sho
11544    Pro
11545    Ent
11546    Pro
11547    Eve
11548    Pro
11549    Ent
Name: First_Tag, Length: 11446, dtype: object

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((9156,), (2290,), (9156,), (2290,))

In [49]:
vect = TfidfVectorizer(stop_words='english')
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

encoder = ce.OrdinalEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

<9156x670 sparse matrix of type '<class 'numpy.float64'>'
	with 599956 stored elements in Compressed Sparse Row format>

In [50]:
sgd = SGDClassifier()

sgd.fit(X_train, y_train)

sgd.score(X_test, y_test)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9462882096069869

In [51]:
rf = RandomForestClassifier()

rf.fit(X_train, y_train)

rf.score(X_test, y_test)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  app.launch_new_instance()


0.9253275109170306

In [155]:
# tfidf = TfidfVectorizer(tokenizer=tokenize, analyzer='word', strip_accents='unicode', min_df=0.025, max_df=.95, ngram_range=(1,2))

# sparse = tfidf.fit_transform(emails['body'])

In [156]:
# dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

# dtm.head()

,aaa,aaaa,aaad,aab,aad,aaecaxeebsex,aaecaxeebsex rcygromjygpkju,aaf,aaf fhbyjxfdkbkaeii,aap,...,zzs,zzs xojig,zzs xojignca,zzxjp,zzxjpzii,zzxjpzjsncgljb,zzxjpzjsncgljb xvcjojmuy,zzxjpzjt,zzxjpzjtjb,zzxjpzjtjb xvcjojmuy
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.016173,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
ALS_test = df.loc[10020]

In [68]:
ALS_test['Text']

'"The ALS Association Florida Chapter" <jniehoff@alsafl.org> Kids Camp Hope Loves Company is coming to Florida in April! =54he ALS Association Florida Chapter=0A___________________________________=\r\n=5F_____________________________=0A=0A=C2=A0Hope Loves Company\'s Camp HLC i=\r\n=73=0A=0AComing to Florida Again this Year!=C2=A0=0A=0A=C2=A0=0A=0AThe Flor=\r\n=69da Chapter is excited to sponsor Hope Loves Company, Inc. to=0Ahelp brin=\r\n=67 Camp HLC to Florida again this fall!=C2=A0 Hope Loves=0ACompany is a no=\r\n=6E profit organization with the mission of providing=0Aboth educational an=\r\n=64 emotional support to children and young adults,=0Aages 6 - 21, who have=\r\n=20or had a loved one battling ALS.=C2=A0 It is an=0Aopportunity to have fu=\r\n=6E, to be challenged through team building=0Aexercises, and to meet other =\r\n=63hildren who are in the same=0Asituation.=C2=A0 =C2=A0=0A=0A=C2=A0=0AThe =\r\n=63amp is free to attendees thanks to supporters like The ALS=0AAssociation=\r